Dream Housing Finance company deals in all kinds of home loans. They have presence across all urban, semi urban and rural areas. Customer first applies for home loan and after that company validates the customer eligibility for loan.

Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have provided a dataset to identify the customers segments that are eligible for loan amount so that they can specifically target these customers. 

Loan_ID:	Unique Loan ID
Gender:	Male/ Female
Married:	Applicant married (Y/N)
Dependents:	Number of dependents
Education:	Applicant Education (Graduate/ Under Graduate)
Self_Employed:	Self employed (Y/N)
ApplicantIncome:	Applicant income
CoapplicantIncome:	Coapplicant income
LoanAmount:	Loan amount in thousands
Loan_Amount_Term:	Term of loan in months
Credit_History:	credit history meets guidelines
Property_Area:	Urban/ Semi Urban/ Rural
Loan_Status	(Target): Loan approved (Y/N)

In [1]:
import numpy as np
import pandas as pd
from sklearn import set_config
set_config(display="diagram")
from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier

Data

In [2]:
df = pd.read_csv('train_data.csv')
X = df.drop(['Loan_Status'], axis=1)
y = df['Loan_Status']

Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
""" 
StratifiedKFold is a variation of k-fold which returns stratified folds: 
each set contains approximately the same percentage of samples of each target class as the complete set.
"""

' \nStratifiedKFold is a variation of k-fold which returns stratified folds: \neach set contains approximately the same percentage of samples of each target class as the complete set.\n'

Data type column separation and corresponding transformers :

Numerical columns

In [4]:
numeric_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

categorical_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

Column preprocessing

In [5]:
preprocessor = make_column_transformer(
    (numeric_transformer, numeric_cols),
    (categorical_transformer, categorical_cols)
)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])
#pipe.fit(X_train, y_train);
#y_pred = pipe.predict(X_test)
#print("Accuracy:", accuracy_score(y_test, y_pred))


In [6]:
from sklearn.model_selection import GridSearchCV

params={
    'clf__n_estimators':[100, 200, 500],
    'clf__max_depth': [5, 6, 7, 8]
}
grid_pipe = GridSearchCV(pipe, param_grid=params, cv=5, verbose=1)

grid_pipe.fit(X_train, y_train)

print(grid_pipe.best_params_)
print(grid_pipe.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
{'clf__max_depth': 6, 'clf__n_estimators': 100}
0.8065965780251494
